# Modeling Wolf Population Dynamics in Yellowstone

This notebook demonstrates several differential equation models for the wolf population in Yellowstone National Park using Python and Matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# For reproducibility
np.random.seed(42)

## 1. Logistic Growth Model
The logistic growth equation models population growth with a carrying capacity $K$.

In [ ]:
def logistic_growth(P, t, r, K):
    return r * P * (1 - P / K)

# Parameters
r = 0.3  # growth rate
K = 500  # carrying capacity
P0 = 50  # initial population
t = np.linspace(0, 40, 400)

P = odeint(logistic_growth, P0, t, args=(r, K))

plt.figure(figsize=(8,5))
plt.plot(t, P, label='Logistic Growth', color='green')
plt.xlabel('Time (years)')
plt.ylabel('Wolf Population')
plt.title('Wolf Population: Logistic Growth')
plt.legend()
plt.grid(True)
plt.show()

# The population rises quickly at first, then levels off as it approaches the carrying capacity.

## 2. Predator-Prey Model (Lotka-Volterra)
This system models interactions between wolves (predators) and elk (prey).

In [ ]:
def lotka_volterra(y, t, a, b, c, d):
    W, E = y
    dWdt = a * W - b * W * E
    dEdt = -c * E + d * W * E
    return [dWdt, dEdt]

# Parameters
a = 0.1   # wolf growth rate
b = 0.01  # predation rate
c = 0.1   # elk death rate
d = 0.005 # elk reproduction per wolf encounter
W0 = 40   # initial wolves
E0 = 800  # initial elk
t2 = np.linspace(0, 80, 800)

sol = odeint(lotka_volterra, [W0, E0], t2, args=(a, b, c, d))
W = sol[:,0]
E = sol[:,1]

plt.figure(figsize=(8,5))
plt.plot(t2, W, label='Wolves', color='red')
plt.plot(t2, E, label='Elk', color='blue', linestyle='--')
plt.xlabel('Time (years)')
plt.ylabel('Population')
plt.title('Lotka-Volterra: Wolves and Elk')
plt.legend()
plt.grid(True)
plt.show()

# The populations oscillate: wolves increase when elk are abundant, then decline, and so on.

## 3. Population Decline Scenario
A sudden drop in carrying capacity (e.g., after a harsh winter) causes the population to adjust.

In [ ]:
K_high = 500
K_low = 200
t3 = np.linspace(0, 40, 400)

def logistic_piecewise(P, t, r, K_high, K_low, t_change):
    K = K_high if t < t_change else K_low
    return r * P * (1 - P / K)

def integrate_piecewise(P0, t, r, K_high, K_low, t_change):
    dt = t[1] - t[0]
    P = [P0]
    for i in range(1, len(t)):
        K = K_high if t[i] < t_change else K_low
        dP = r * P[-1] * (1 - P[-1] / K)
        P.append(P[-1] + dP * dt)
    return np.array(P)

t_change = 15
P3 = integrate_piecewise(P0, t3, r, K_high, K_low, t_change)

plt.figure(figsize=(8,5))
plt.plot(t3, P3, label='Sudden Drop in K', color='purple')
plt.axvline(t_change, color='gray', linestyle=':', label='Resource Shock')
plt.xlabel('Time (years)')
plt.ylabel('Wolf Population')
plt.title('Population Decline After Resource Loss')
plt.legend()
plt.grid(True)
plt.show()

# The population drops after the carrying capacity decreases, then stabilizes at the new lower K.

## 4. Human Interference Scenario
Simulate human-caused removals (e.g., hunting or relocation) by adding a constant removal rate $h$.

In [ ]:
def logistic_human(P, t, r, K, h):
    return r * P * (1 - P / K) - h

h = 10  # wolves removed per year
P4 = odeint(logistic_human, P0, t, args=(r, K, h))

plt.figure(figsize=(8,5))
plt.plot(t, P, label='No Human Interference', color='green', linestyle='--')
plt.plot(t, P4, label='With Human Removal', color='orange')
plt.xlabel('Time (years)')
plt.ylabel('Wolf Population')
plt.title('Wolf Population: Human Interference')
plt.legend()
plt.grid(True)
plt.show()

# Human-caused removal lowers the equilibrium population, or can even cause extinction if h is too high.